# Data collection

In [1]:
import pandas as pd
from faker import Faker

import fetch
import modules.urls as urls
from fetch.utils import json_or_fetch

In [2]:
df = pd.read_excel('data/SOFT-3.xlsx', header=1)
df = df.drop(df.columns[0], axis=1)
# df.head()

## Anonymising the data (student names)

In [3]:
faker = Faker()
dict_names = {name: faker.name() for name in df['student'].unique()}
df['student'] = df['student'].map(dict_names)
df.head()

,student,company
0,Jimmy Bell,Dynatest A/S
1,David Torres,"Eriksholm Research Centre, Oticon"
2,Mariah Williams,Formpipe
3,Tara Benjamin,Formpipe
4,Sheila Tucker,Formpipe


## Getting company data with CVR API

In [4]:
df['company'].unique().size

25

These are the older deprecated cells that fetched and stored JSON data about the companies. There's a refactored version meant to be more reusable in `fetch/utils.py`.

In [5]:
# import json
# from os.path import exists

# # init empty dict
# dict_company_data = {}
# # loads from json file if exists
# path = 'data/company_data.json'
# if exists(path):
#     with open(path) as f:
#         dict_company_data = json.load(f)

In [6]:
# # if company from df is not in the existing json file, we get info from API
# for company in df['company'].unique():
#     if company not in dict_company_data.keys():
#         dict_company_data[company] = get_company_data(company)

# # overwrite existing json file with fresh data
# with open(path, 'w') as f:
#     json.dump(dict_company_data, f)

In [7]:
keys = df['company'].unique()
args = tuple(zip(keys))

In [8]:
dict_company_data = json_or_fetch(fetch.cvrapi, keys, args, path='data/company_data.json')

In [9]:
df_company = pd.DataFrame.from_dict(dict_company_data, orient='index')
df_company.head()

,vat,name,address,zipcode,city,cityname,protected,phone,email,fax,...,industrydesc,companycode,companydesc,creditstartdate,creditbankrupt,creditstatus,owners,productionunits,t,version
Dynatest A/S,20318872,A/S DYNATEST ENGINEERING,Generatorvej 21,2730,Herlev,None,False,NaN,None,NaN,...,Uoplyst,60,Aktieselskab,12/03 - 2001,True,1.0,None,"[{'pno': 1004239898, 'main': True, 'name': 'A/...",100,6
"Eriksholm Research Centre, Oticon",50091015,PROPOLIS RESEARCH CENTRE A/S,Lille Veksøvej 11,3670,Veksø Sjælland,None,False,NaN,None,NaN,...,"Andre liberale, videnskabelige og tekniske tje...",60,Aktieselskab,None,False,NaN,None,"[{'pno': 1001942680, 'main': True, 'name': 'PR...",100,6
Formpipe,26366216,FORMPIPE LASERNET A/S,Lautrupvang 1,2750,Ballerup,None,False,43660210.0,admin.lasernet@formpipe.com,NaN,...,Anden udgivelse af software,60,Aktieselskab,None,False,NaN,[{'name': 'Formpipe Software AB'}],"[{'pno': 1008829868, 'main': True, 'name': 'FO...",100,6
Novo Nordisk,24256790,NOVO NORDISK A/S,Novo Alle 1,2880,Bagsværd,None,False,44448888.0,None,NaN,...,Fremstilling af farmaceutiske præparater,60,Aktieselskab,None,False,NaN,None,"[{'pno': 1017661031, 'main': False, 'name': 'N...",100,6
PFA,31050162,PFA BANK A/S,Sundkrogsgade 4,2100,København Ø,None,False,70808500.0,post@pfabank.dk,39172950.0,...,"Banker, sparekasser og andelskasser",60,Aktieselskab,None,False,NaN,None,"[{'pno': 1013750293, 'main': True, 'name': 'PF...",100,6


## Get more data from Proff
Because we don't have access to the API, and there's no way that I can see of deducing the URL to the company page, I will just make a `list` of URLs to scrape, one for each company. The annoying part is having to update this when new companies are added. Same goes for the URLs below when we do text vectorisation.

If we had the most accurate CVR number in our starting data, we could search on that programmatically, and pick the top result. As it is, we could do that, but if CVR API gave us the wrong company based on only the name, we won't get the most accurate data.

**Actually!** You can use the URL pattern `https://proff.dk/firma/-/-/-/CVR`. It'll redirect to the real page. The `-` (dashes) are also optional, but whenever Proff themselves shorten the links, they put those in as placeholders. This way we can get the links completely programatically more easily.

Another problem is what to do with companies you do not have their finances publically available on Proff. They should be all NaN, then. It'll either error when you try to get a table from the empty site, or if you try to find the link from the main which doesn't exist (no `<a>` tag). So how you error handle might change over time.

In [10]:
keys = urls.proff
args = tuple(zip(keys))

In [11]:
proff_data = json_or_fetch(fetch.proff, keys, args, path='data/proff_data.json')

Now that we have the data from Proff, we can add it to our dataframe.

In [12]:
# from_dict has a messed up row order, so we transpose it
df_proff = pd.DataFrame(proff_data).T
df_proff.set_index(df_company.index, inplace=True)
df_proff

,2022,2021,2020,employees,2019,2018,2017,2016,2015,2014,2013
Dynatest A/S,3581.0,9606.0,4530.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Eriksholm Research Centre, Oticon",NaN,1525319.0,165416.0,42.0,551594.0,962573.0,1011568.0,743559.0,730954.0,535275.0,798601.0
Formpipe,NaN,36583.0,14719.0,87.0,7525.0,10511.0,-748.0,-4924.0,-4145.0,-590.0,-7751.0
Novo Nordisk,NaN,47523000.0,42159000.0,22150.0,39252000.0,38530000.0,37873000.0,37791000.0,34623000.0,26413000.0,25190000.0
PFA,-511000.0,-79000.0,108000.0,1464.0,10000.0,92000.0,181000.0,136000.0,-587000.0,548000.0,224000.0
Topdanmark,NaN,2138000.0,NaN,8.0,1547000.0,1331000.0,1733000.0,1536000.0,1132000.0,1558000.0,1468000.0
KMD A/S,53926.0,2893.0,173758.0,1855.0,-238596.0,NaN,177700.0,244400.0,500600.0,81800.0,53100.0
NorthTech ApS,NaN,5.0,598.0,20.0,792.0,1124.0,1010.0,1497.0,1931.0,672.0,362.0
Alpha Solutions,3477.0,4334.0,-2940.0,59.0,-3279.0,4171.0,3385.0,2243.0,5079.0,3439.0,2500.0
Dafolo,36271.0,1339.0,40.0,83.0,3162.0,-489.0,-5862.0,-2121.0,1439.0,4170.0,844.0


The column order is slightly messed up because employees is encountered before the rest of the years, so we could move it to the front with `pop` and `insert`.

```python
df_proff.insert(0, 'employees', df_proff.pop('employees'))
```

However, we actually want to just replace the original column.

In [13]:
df_company['employees'] = df_proff.pop('employees')

In [14]:
# cast columns names (years) to int
# annoyingly, when we read these headers from csv, they are read as strings anyway.
# and then it's much harder to locate them to cast them.
# so it might be better to not cast it here, and just cast it when we need it.

# df_proff.columns = df_proff.columns.astype(int)

In [15]:
df_company = pd.concat([df_company, df_proff], axis=1)

In [16]:
df_company.to_csv('data/1_company_data.csv')